# Установка нужных библиотек

# Импорт библиотек

In [1]:
import os
import json
import glob
import torch

import matplotlib.pyplot as plt

from src.mylib.models.models import BaselineModel, MHAModel
from src.mylib.train import Trainer

file = os.path.abspath('')

2024-04-01 20:21:47.470487: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-01 20:21:48.253379: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Работа с данными

Начальные параметры

In [2]:
batch_size = 64

# Длина окна
window_length_seconds = 5 
sample_rate = 64
window_length = window_length_seconds * sample_rate

# Расстояние между двумя окнами
hop_length_seconds = 1
hop_length = sample_rate * hop_length_seconds

# Количество ложные стимулов
number_of_mismatch = 4

In [3]:
experiment_folder = os.path.dirname(file)

# Load the config file
with open(os.path.join(experiment_folder, "src/mylib/utils/config.json")) as file_path:
    config = json.load(file_path)

# Path to the dataset, which is already split to train, val, test
data_folder = os.path.join(config["dataset_folder"], config['derivatives_folder'], config["split_folder"])

# Пути к данным тренировочным, валидационным и тестовым данным
train_files = [x for x in glob.glob(os.path.join(data_folder, "train_-_*")) if
                       os.path.basename(x).split("_-_")[-1].split(".")[0] in ["eeg", "envelope"]]
val_files = [x for x in glob.glob(os.path.join(data_folder, "val_-_*")) if
                       os.path.basename(x).split("_-_")[-1].split(".")[0] in ["eeg", "envelope"]]
test_files = [x for x in glob.glob(os.path.join(data_folder, "test_-_*")) if
                       os.path.basename(x).split("_-_")[-1].split(".")[0] in ["eeg", "envelope"]]

## Обучение 

Базовое решение

In [4]:
# model = BaselineModel()
args = {"window_length" : window_length, "hop_length" : hop_length, "number_of_mismatch" : number_of_mismatch, "batch_size" : batch_size, 
        "max_files" : 100}
for model in [BaselineModel, MHAModel]:
    model = model()
    print(f"Model: {model.__class__.__name__}")
    
    trainer = Trainer(
        model, train_files, val_files, test_files, args, torch.optim.Adam(model.parameters(), lr=0.001), torch.nn.CrossEntropyLoss(), 
    )

    trainer.train_model(epochs=5, run_name=f"{model.__class__.__name__}")
    print(trainer.test())
    print("-----" * 5)

EPOCH 1:
  batch 100 loss: 0.1482216489744413
  batch 200 loss: 0.0002496870611862123
  batch 300 loss: 0.0004293969416104437
  batch 400 loss: 0.00014352307940049714
  batch 500 loss: 0.0003824394412247301
  batch 600 loss: 0.0005175532017259399
  batch 700 loss: 0.00033389098716623877
  batch 800 loss: 8.32907846506714e-05
  batch 900 loss: 2.4057848494294377e-05
LOSS train 2.4057848494294377e-05 valid 0.0
EPOCH 2:
  batch 100 loss: 2.239500692658325e-05
  batch 200 loss: 1.717424765956821e-07
  batch 300 loss: 2.382965994911501e-06
  batch 400 loss: 2.0097729375834205e-07
  batch 500 loss: 5.319957272149622e-06
  batch 600 loss: 4.880248161498457e-07
  batch 700 loss: 7.180011834861944e-08
  batch 800 loss: 1.2190427505487378e-06
  batch 900 loss: 2.3094077732821461e-07
LOSS train 2.3094077732821461e-07 valid 0.0
EPOCH 3:
  batch 100 loss: 1.9006388822617737e-06
  batch 200 loss: 1.8029883719350436e-08
  batch 300 loss: 5.650982788552028e-07
  batch 400 loss: 3.155075988914291e-08
 